<a href="https://colab.research.google.com/github/abishekP101/Pytorch/blob/main/Autograd_and_trainingPipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
x = torch.tensor(3.0 , requires_grad=True)

In [ ]:
y = x**2


In [ ]:
print(x , y)

tensor(3., requires_grad=True) tensor(9., grad_fn=<PowBackward0>)


In [ ]:
y.backward()

In [ ]:
x.grad

tensor(6.)

In [ ]:
x = torch.tensor(3.0 , requires_grad=True)
y = x**2
z = torch.sin(y)

In [ ]:
print(x , y , z)

tensor(3., requires_grad=True) tensor(9., grad_fn=<PowBackward0>) tensor(0.4121, grad_fn=<SinBackward0>)


In [ ]:
x.grad

tensor(-5.4668)

In [ ]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

w = torch.tensor(1.0 , requires_grad=True)
b = torch.tensor(0.0 , requires_grad=True)



In [ ]:
def binary_cross_entropy_loss(prediction , target):
  epsilon = 1e-8
  prediction = torch.clamp(prediction , epsilon ,  1-epsilon)
  return -(target*torch.log(prediction)+(1-target)*torch.log(1-prediction))

In [ ]:
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [ ]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [ ]:
loss = binary_cross_entropy_loss(y_pred , y)

In [ ]:
loss.backward()

In [ ]:
print(w.grad , b.grad)

tensor(6.6918) tensor(0.9988)


In [ ]:
x = torch.tensor([1.0 , 2.0 , 3.0] , requires_grad=True)
x

tensor([1., 2., 3.], requires_grad=True)

In [ ]:
y  = (x**2).mean()

In [ ]:
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [ ]:
y.backward()

In [ ]:
x.grad

tensor([0.6667, 1.3333, 2.0000])

## Pytorch Training Pipeline

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


In [ ]:
df  = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
df.shape

(569, 33)

In [ ]:
df.drop(columns=['Unnamed: 32' , 'id'] , inplace=True)

In [ ]:
X = df.drop(columns=['diagnosis'])
y = df['diagnosis']

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
label = LabelEncoder()
y_train = label.fit_transform(y_train)
y_test = label.transform(y_test)

In [ ]:
X_train_tensor = torch.tensor(X_train , dtype=torch.float32)
X_test_tensor = torch.tensor(X_test , dtype=torch.float32)
y_train_tensor = torch.tensor(y_train , dtype=torch.float32)
y_test_tensor = torch.tensor(y_test , dtype=torch.float32)

In [ ]:
class MySimpleNN():
  def __init__(self , x):
    self.weights = torch.randn(x.shape[1] , 1 , dtype=torch.float32 , requires_grad=True)
    self.bias = torch.randn(1 , requires_grad=True)

  def forward(self , x):
    z = torch.matmul(x , self.weights) + self.bias
    return torch.sigmoid(z)

  def loss_function(self , y_pred , y):
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred , epsilon , 1-epsilon)
    return -(y*torch.log(y_pred) + (1-y)*torch.log(1-y_pred)).mean()

## Important Parameters

In [ ]:
learning_rate= 0.1
epochs = 25

In [ ]:
model = MySimpleNN(X_train_tensor)

for epoch in range(epochs):
  y_pred = model.forward(X_train_tensor)
  loss = model.loss_function(y_pred , y_train_tensor)
  loss.backward()
  with torch.no_grad():
    model.weights -= learning_rate*model.weights.grad
    model.bias -= learning_rate*model.bias.grad


  model.weights.grad.zero_()
  model.bias.grad.zero_()

  print(f'Epoch : {epoch+1} , Loss : {loss.item()}')


Epoch : 1 , Loss : 2.358886241912842
Epoch : 2 , Loss : 2.2936019897460938
Epoch : 3 , Loss : 2.227064371109009
Epoch : 4 , Loss : 2.158198356628418
Epoch : 5 , Loss : 2.0895049571990967
Epoch : 6 , Loss : 2.022489070892334
Epoch : 7 , Loss : 1.9570224285125732
Epoch : 8 , Loss : 1.893554925918579
Epoch : 9 , Loss : 1.8317238092422485
Epoch : 10 , Loss : 1.7687580585479736
Epoch : 11 , Loss : 1.70841646194458
Epoch : 12 , Loss : 1.650291919708252
Epoch : 13 , Loss : 1.5946601629257202
Epoch : 14 , Loss : 1.5415188074111938
Epoch : 15 , Loss : 1.4908742904663086
Epoch : 16 , Loss : 1.442736268043518
Epoch : 17 , Loss : 1.3970916271209717
Epoch : 18 , Loss : 1.3539223670959473
Epoch : 19 , Loss : 1.3132158517837524
Epoch : 20 , Loss : 1.274959683418274
Epoch : 21 , Loss : 1.2391427755355835
Epoch : 22 , Loss : 1.2057478427886963
Epoch : 23 , Loss : 1.1747478246688843
Epoch : 24 , Loss : 1.1460998058319092
Epoch : 25 , Loss : 1.1197404861450195


In [ ]:
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  loss = model.loss_function(y_pred , y_test_tensor)
  print(f'Test Loss : {loss.item()}')

Test Loss : 1.0682611465454102


In [ ]:
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred>0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Test Accuracy : {accuracy.item()}')

Test Accuracy : 0.5947983860969543
